### Traditional Model Solution

In [1]:
import math
import numpy
from gurobipy import *
import os

'''
supply chain configuration
'''
c = 0
h_List = [182,660,1155,465,46,218,270,345,1292,2235,2770,1350,195,2129,8697,2775,172,27,900,17201,1279,21780]
b_List = [1000,1000,1000,1000,1000,1000,1000,1000,5000,1000,5000,1000,1000,5000,10000,5000,1000,1000,1000,20000,5000,40000]
L_List = [19,15,8,9,9,6,9,8,16,15,6,7,2,7,5,10,11,35,10,8,7,4]

N = 22
Structure = [[-1],[-1],[-1],[-1],[-1],[-1],[-1],[-1],[0,1],[-1],[2,3,4],[-1],[-1],[5,6,7],[8,9,10],[11,12],[-1],[-1],[-1],[13,14,15],[16,17],[18,19,20]]
S_in_supplier = [10,10,10,10,10,10,10,10,1e20,10,1e20,10,10,1e20,1e20,1e20,10,10,10,1e20,1e20,1e20]
S_out_customer = [1e20,1e20,1e20,1e20,1e20,1e20,1e20,1e20,1e20,1e20,1e20,1e20,1e20,1e20,1e20,1e20,1e20,1e20,1e20,1e20,1e20,5]

Non_Start = [] #find the set of stages with upstream stages
for j in range (N):
    if (Structure[j][0] > -1):
        Non_Start.append(j)

for i in range (N): #re-scaling
    b_List[i] = b_List[i] *10 / 100.0
    h_List[i] = h_List[i] / 100.0

'''
distribution used in model formulation
'''
probs_Z = [0.33,0.34,0.33]
values_Z = [1,2,3] #Z
mu_model_Z = 0
mu_sq_model_Z= 0
mu_model = 0 #demand mean
mu_sq_model= 0
for pre_i in range (len(probs_Z)):
    mu_model_Z += probs_Z[pre_i]*values_Z[pre_i]
    mu_sq_model_Z += probs_Z[pre_i]*values_Z[pre_i]*values_Z[pre_i]
    mu_model += probs_Z[pre_i]/values_Z[pre_i]
    mu_sq_model += probs_Z[pre_i]/values_Z[pre_i]/values_Z[pre_i]
sigma_Z = math.sqrt(mu_sq_model_Z-mu_model_Z*mu_model_Z)
sigma = math.sqrt(mu_sq_model-mu_model*mu_model) #demand std

'''
calculate optimal solution
'''
S_MAX = 101
           
def optimal_solution (k):
    F_S_S = []
    solution = []
    for S_in in range (0,S_MAX):
        temp = []
        for S_out in range (0,S_MAX):
            SubCost = []
            for j in range (N):
                if ((S_in+L_List[j]-S_out) >= 0):
                    SubCost.append(h_List[j]*k*sigma*math.sqrt(S_in+L_List[j]-S_out))
                else: 
                    SubCost.append(5000)
            temp.append(SubCost)
        F_S_S.append(temp)

    F_S_S = numpy.array(F_S_S)
    print("F_S_S generated!")

    #Solve MILP
    try:
        m = Model("Algo-Trad")
        # Create variables
        chi = m.addVars(N, S_MAX, S_MAX, vtype=GRB.BINARY, name="chi")
        # Set objective
        obj = 0
        for j in range (N):
            for mu in range (S_MAX):
                for nu in range (S_MAX):
                    obj += F_S_S[mu,nu,j]*chi[j,mu,nu]
        m.setObjective(obj, GRB.MINIMIZE)
        # Add constraint
        for j in range (N):
            m.addConstr(quicksum(chi[j,mu,nu] for mu in range (S_MAX) for nu in range (S_MAX)) == 1)
            if (S_out_customer[j] < 1e20):
                m.addConstr(quicksum(nu*chi[j,mu,nu] for mu in range (S_MAX) for nu in range (S_MAX)) == S_out_customer[j])
            if (S_in_supplier[j] < 1e20):
                m.addConstr(quicksum(mu*chi[j,mu,nu] for mu in range (S_MAX) for nu in range (S_MAX)) == S_in_supplier[j])
        for i in range (len(Non_Start)):
            for j in Structure[Non_Start[i]]:
                m.addConstr(quicksum(mu*chi[Non_Start[i],mu,nu] for mu in range (S_MAX) for nu in range (S_MAX)) >= quicksum(nu*chi[j,mu,nu] for mu in range (S_MAX) for nu in range (S_MAX)))

        m.optimize()

    except GurobiError as e:
        print('Error code ' + str(e.errno) + ': ' + str(e))

    except AttributeError:
        print('Encountered an attribute error')

    for j in range (N):
        for mu in range (S_MAX):
            for nu in range (S_MAX):
                if(chi[j,mu,nu].X == 1):
                    solution.append([F_S_S[mu,nu,j],mu,nu]) #obj,S_in,S_out
    
    return solution.copy()

Trad_General = open("Trad-General-Simulation.txt", mode='w')
Trad_Poisson = open("Trad-Poisson-Simulation.txt", mode='w')
Trad_Solution = open("Trad-Solution.txt", mode='w')
for k in [1.0,1.1,1.2,1.3,1.4,1.5,1.6,1.7,1.8,1.9,2.0,
         2.1,2.2,2.3,2.4,2.5,2.6,2.7,2.8,2.9,3.0,
          3.1,3.2,3.3,3.4,3.5,3.6,3.7,3.8,3.9,4.0,
         4.1,4.2,4.3,4.4,4.5,4.6,4.7,4.8,4.9,5.0]:
    opt_temp = optimal_solution(k)
    opt_temp = numpy.array(opt_temp)
    S_in_List = opt_temp[:,1]
    S_out_List = opt_temp[:,2]
    S_in_List = S_in_List.tolist()
    S_out_List = S_out_List.tolist()
    W_List = S_in_List
    B_List = []
    for i in range (N):
        B_List.append(round(mu_model*(S_in_List[i]+L_List[i]-S_out_List[i])+k*sigma*math.sqrt(S_in_List[i]+L_List[i]-S_out_List[i]))) 
    Trad_Solution.write(str(k)+'\t')
    for i in range (N):
        Trad_Solution.write(str(S_in_List[i])+'\t')
    for i in range (N):
        Trad_Solution.write(str(S_out_List[i])+'\t')
    for i in range (N):
        Trad_Solution.write(str(B_List[i])+'\t')
    Trad_Solution.write('\n') 
    #simulation-general
    for w in range (100):
        numpy.random.seed(w+10)
        Z = numpy.random.choice(values_Z,500,p=probs_Z).tolist() #Z
        demand = [] #real-time demand 0 or 1
        for z in Z:
            for z_i in range (z-1):
                demand.append(0)
            demand.append(1)
        cost = numpy.zeros(N).tolist()
        backordertimes = numpy.zeros(N).tolist()
        for i in range (N):
            stock = []
            stock.append(B_List[i])
            for j in range (1, len(demand)):
                stock.append(stock[j-1])
                if ((j-S_out_List[i]) >= 0):
                    stock[j] -= demand[int(j-S_out_List[i])]
                if ((j-W_List[i]-L_List[i]) >= 0):
                    stock[j] += demand[int(j-W_List[i]-L_List[i])]
                if (stock[j] < 0):
                    cost[i] -= stock[j]*b_List[i]
                    backordertimes[i] -= stock[j]
                    stock[j] = 0
                else:
                    cost[i] += h_List[i]*stock[j]
        Trad_General.write(str(k)+'\t'+str(w)+'\t')
        for i in range (N):
            Trad_General.write(str(cost[i])+'\t')
        for i in range (N):
            Trad_General.write(str(backordertimes[i])+'\t')
        Trad_General.write('\n')
    print(k, "Simulation-General Ended!")
    #simulation-poisson
    for w in range (100):
        numpy.random.seed(w+10)
        demand = numpy.random.poisson(mu_model,int(500*mu_model_Z)).tolist()
        cost = numpy.zeros(N).tolist()
        backordertimes = numpy.zeros(N).tolist()
        for i in range (N):
            stock = []
            stock.append(B_List[i])
            for j in range (1, len(demand)):
                stock.append(stock[j-1])
                if ((j-S_out_List[i]) >= 0):
                    stock[j] -= demand[int(j-S_out_List[i])]
                if ((j-W_List[i]-L_List[i]) >= 0):
                    stock[j] += demand[int(j-W_List[i]-L_List[i])]
                if (stock[j] < 0):
                    cost[i] -= stock[j]*b_List[i]
                    backordertimes[i] -= stock[j]
                    stock[j] = 0
                else:
                    cost[i] += h_List[i]*stock[j]
        Trad_Poisson.write(str(k)+'\t'+str(w)+'\t')
        for i in range (N):
            Trad_Poisson.write(str(cost[i])+'\t')
        for i in range (N):
            Trad_Poisson.write(str(backordertimes[i])+'\t')
        Trad_Poisson.write('\n')
    print(k, "Simulation-Poisson Ended!") 
Trad_General.close() #every row: k,w,cost of stages 0~21, backorder times of stages 0~21
Trad_Poisson.close()
Trad_Solution.close() #every row: k, inbound service time of stages 0~21, outbound service time of stages 0~21, base stock of stages 0~21

F_S_S generated!

--------------------------------------------
--------------------------------------------

Academic license - for non-commercial use only - expires 2021-07-13
Using license file /Users/estelle/gurobi.lic
Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (mac64)
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 58 rows, 224422 columns and 800122 nonzeros
Model fingerprint: 0x1dd404c9
Variable types: 0 continuous, 224422 integer (224422 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+02]
  Objective range  [8e-02, 5e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+01]
Found heuristic solution: objective 891.6229150
Presolve removed 15 rows and 199464 columns (presolve time = 6s) ...
Presolve removed 4 rows and 200514 columns
Presolve time: 5.77s
Presolved: 54 rows, 23908 columns, 113988 nonzeros
Found heuristic solution: objective 729.7780677
Variable types: 0 continuous, 23908 integer (23908


Cutting planes:
  Gomory: 13
  Cover: 5
  Clique: 5
  MIR: 7
  StrongCG: 6
  GUB cover: 18
  Zero half: 5

Explored 1 nodes (682 simplex iterations) in 7.45 seconds
Thread count was 4 (of 4 available processors)

Solution count 9: 356.884 358.202 404.084 ... 980.785

Optimal solution found (tolerance 1.00e-04)
Best objective 3.568838573170e+02, best bound 3.568838573170e+02, gap 0.0000%
1.1 Simulation-General Ended!
1.1 Simulation-Poisson Ended!
F_S_S generated!
Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (mac64)
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 58 rows, 224422 columns and 800122 nonzeros
Model fingerprint: 0x3cbe4be2
Variable types: 0 continuous, 224422 integer (224422 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+02]
  Objective range  [9e-02, 5e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+01]
Found heuristic solution: objective 1069.9474980
Presolve removed 15 rows and 199464

     0     0  420.22356    0   51  421.77183  420.22356  0.37%     -    9s
     0     0  420.36185    0   53  421.77183  420.36185  0.33%     -    9s
     0     0  420.41940    0   56  421.77183  420.41940  0.32%     -    9s
     0     0     cutoff    0       421.77183  421.77183  0.00%     -    9s

Cutting planes:
  Cover: 5
  Clique: 6
  MIR: 9
  StrongCG: 4
  GUB cover: 20
  Zero half: 8
  RLT: 2

Explored 1 nodes (650 simplex iterations) in 9.80 seconds
Thread count was 4 (of 4 available processors)

Solution count 10: 421.772 422.487 424.422 ... 948.711

Optimal solution found (tolerance 1.00e-04)
Best objective 4.217718313746e+02, best bound 4.217718313746e+02, gap 0.0000%
1.3 Simulation-General Ended!
1.3 Simulation-Poisson Ended!
F_S_S generated!
Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (mac64)
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 58 rows, 224422 columns and 800122 nonzeros
Model fingerprint: 0xa556f7ec
Variable

  Matrix range     [1e+00, 1e+02]
  Objective range  [1e-01, 5e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+01]
Found heuristic solution: objective 1426.5966640
Presolve removed 15 rows and 199464 columns (presolve time = 7s) ...
Presolve removed 4 rows and 200514 columns
Presolve time: 7.31s
Presolved: 54 rows, 23908 columns, 113988 nonzeros
Found heuristic solution: objective 1167.6449082
Variable types: 0 continuous, 23908 integer (23908 binary)

Root simplex log...

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   2.100000e+01   0.000000e+00      8s
      85    3.3340758e+02   0.000000e+00   0.000000e+00      8s

Root relaxation: objective 3.334076e+02, 85 iterations, 0.03 seconds
Total elapsed time = 7.70s

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0  333.40758    0   30 1167.64491  333.40758  71.

Variable types: 0 continuous, 23908 integer (23908 binary)

Root simplex log...

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   2.100000e+01   0.000000e+00      8s
      85    3.7508352e+02   0.000000e+00   0.000000e+00      8s

Root relaxation: objective 3.750835e+02, 85 iterations, 0.03 seconds
Total elapsed time = 8.36s

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0  375.08352    0   30 1313.60052  375.08352  71.4%     -    8s
H    0     0                     898.2487555  375.08352  58.2%     -    8s
H    0     0                     715.1617715  375.08352  47.6%     -    8s
H    0     0                     705.2108592  375.08352  46.8%     -    8s
     0     0  436.77043    0   34  705.21086  436.77043  38.1%     -    8s
H    0     0                     661.4004497  436.77043  34.0%     -    8s
     0     0  450.16511    0

...

     0     0 1327.55721    0    2 1330.20347 1327.55721  0.20%     -   10s

Cutting planes:
  Cover: 2
  Clique: 12
  MIR: 1
  StrongCG: 1
  GUB cover: 12
  RLT: 1

Explored 1 nodes (523 simplex iterations) in 11.03 seconds
Thread count was 4 (of 4 available processors)

Solution count 9: 1330.2 1331.46 1506.13 ... 3655.65

Optimal solution found (tolerance 1.00e-04)
Best objective 1.330203468181e+03, best bound 1.330203468181e+03, gap 0.0000%
4.1 Simulation-General Ended!
4.1 Simulation-Poisson Ended!
F_S_S generated!
Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (mac64)
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 58 rows, 224422 columns and 800122 nonzeros
Model fingerprint: 0xf28768e2
Variable types: 0 continuous, 224422 integer (224422 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+02]
  Objective range  [3e-01, 5e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+01]
Found heuristic solution: o

4.3 Simulation-Poisson Ended!
F_S_S generated!
Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (mac64)
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 58 rows, 224422 columns and 800122 nonzeros
Model fingerprint: 0x7286544e
Variable types: 0 continuous, 224422 integer (224422 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+02]
  Objective range  [3e-01, 5e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+01]
Found heuristic solution: objective 3923.1408259
Presolve removed 15 rows and 199539 columns (presolve time = 5s) ...
Presolve removed 4 rows and 200514 columns
Presolve time: 5.15s
Presolved: 54 rows, 23908 columns, 113988 nonzeros
Found heuristic solution: objective 3211.0234977
Variable types: 0 continuous, 23908 integer (23908 binary)

Root simplex log...

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   2.100000e+01   0.000000e+00      5s
      85    9.

     0     0 1459.68932    0    2 1459.97942 1459.68932  0.02%     -    6s

Cutting planes:
  Gomory: 13
  Cover: 6
  Clique: 1
  MIR: 4
  StrongCG: 4
  GUB cover: 14
  Zero half: 4
  RLT: 2

Explored 1 nodes (636 simplex iterations) in 6.55 seconds
Thread count was 4 (of 4 available processors)

Solution count 10: 1459.98 1462.45 1469.15 ... 3284

Optimal solution found (tolerance 1.00e-04)
Best objective 1.459979416297e+03, best bound 1.459979416297e+03, gap 0.0000%
4.5 Simulation-General Ended!
4.5 Simulation-Poisson Ended!
F_S_S generated!
Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (mac64)
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 58 rows, 224422 columns and 800122 nonzeros
Model fingerprint: 0x29299ac0
Variable types: 0 continuous, 224422 integer (224422 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+02]
  Objective range  [4e-01, 5e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+01]
Fo

  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+01]
Found heuristic solution: objective 4279.7899919
Presolve removed 15 rows and 200525 columns (presolve time = 5s) ...
Presolve removed 4 rows and 200514 columns
Presolve time: 5.11s
Presolved: 54 rows, 23908 columns, 113988 nonzeros
Found heuristic solution: objective 3502.9347247
Variable types: 0 continuous, 23908 integer (23908 binary)

Root simplex log...

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   2.100000e+01   0.000000e+00      5s
      85    1.0002227e+03   0.000000e+00   0.000000e+00      5s

Root relaxation: objective 1.000223e+03, 85 iterations, 0.02 seconds
Total elapsed time = 5.41s

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 1000.22273    0   30 3502.93472 1000.22273  71.4%     -    5s
H    0     0                    2395.3300147 1000.222

Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 58 rows, 224422 columns and 800122 nonzeros
Model fingerprint: 0x840805ac
Variable types: 0 continuous, 224422 integer (224422 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+02]
  Objective range  [4e-01, 5e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+01]
Found heuristic solution: objective 4458.1145749
Presolve removed 15 rows and 199464 columns (presolve time = 5s) ...
Presolve removed 4 rows and 200514 columns
Presolve time: 5.65s
Presolved: 54 rows, 23908 columns, 113988 nonzeros
Found heuristic solution: objective 3648.8903383
Variable types: 0 continuous, 23908 integer (23908 binary)

Root simplex log...

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   2.100000e+01   0.000000e+00      6s
      85    1.0418987e+03   0.000000e+00   0.000000e+00      6s

Root relaxation: objective 1.041899e+03, 85 iterati

In [4]:
import os
import numpy

N = 22

File = open("Trad-Solution.txt", mode='r')
temp = []
while True:
    Line = File.readline()
    if Line:
        Line = Line[:-2]
        Line = list(map(float,Line.split('\t')))
        Line = Line[1:]
        temp.append(Line)
    else:
        break
File.close()
print("inbound service time")
print(numpy.average(temp, axis=0)[0:N])
print(numpy.max(temp, axis=0)[0:N])
print(numpy.min(temp, axis=0)[0:N])
print("outbound service time")
print(numpy.average(temp, axis=0)[N:(2*N)])
print(numpy.max(temp, axis=0)[N:(2*N)])
print(numpy.min(temp, axis=0)[N:(2*N)])
print("base stock")
print(numpy.average(temp, axis=0)[(2*N):(3*N)])
print(numpy.max(temp, axis=0)[(2*N):(3*N)])
print(numpy.min(temp, axis=0)[(2*N):(3*N)])

inbound service time
[10. 10. 10. 10. 10. 10. 10. 10. 25. 10.  0. 10. 10.  4.  6.  1. 10. 10.
 10. 11.  0.  1.]
[10. 10. 10. 10. 10. 10. 10. 10. 25. 10.  0. 10. 10.  4.  6.  1. 10. 10.
 10. 11.  0.  1.]
[10. 10. 10. 10. 10. 10. 10. 10. 25. 10.  0. 10. 10.  4.  6.  1. 10. 10.
 10. 11.  0.  1.]
outbound service time
[25. 25.  0.  0.  0.  4.  4.  4.  6.  6.  6.  1.  1. 11. 11. 11.  0.  0.
  1.  1.  1.  5.]
[25. 25.  0.  0.  0.  4.  4.  4.  6.  6.  6.  1.  1. 11. 11. 11.  0.  0.
  1.  1.  1.  5.]
[25. 25.  0.  0.  0.  4.  4.  4.  6.  6.  6.  1.  1. 11. 11. 11.  0.  0.
  1.  1.  1.  5.]
base stock
[ 4.12195122  0.         14.56097561 15.29268293 15.29268293 10.24390244
 12.43902439 11.73170732 26.41463415 15.29268293  0.         13.09756098
  9.51219512  0.          0.          0.         16.65853659 33.14634146
 15.29268293 14.56097561  5.73170732  0.        ]
[ 5.  0. 17. 18. 18. 12. 15. 14. 30. 18.  0. 15. 11.  0.  0.  0. 19. 37.
 18. 17.  7.  0.]
[ 3.  0. 12. 13. 13.  8. 10. 10. 23. 13.